In [1]:
import os
import glob

wsj ="./dataset/trec/wsj"
ap ="./dataset/trec/ap"
wsj_files = glob.glob(wsj+"/*")
ap_files = glob.glob(ap+"/*")

docs_json = {}
errors = 0
for wsj_file in wsj_files:
    with open(wsj_file,'r') as f:
        txt = f.read()
    docs = [i.split("<DOC>")[1] for i in filter(lambda x:len(x)>10,txt.split('</DOC>'))]

    for doc in docs:
        try:
            id = doc.split("<DOCNO>")[1].split('</DOCNO>')[0]
            text = doc.split("<TEXT>")[1].split('</TEXT>')[0]
            docs_json[id] = text
        except:
            errors+=1

for ap_file in ap_files:
    with open(ap_file,'r',encoding="latin-1") as f:
        txt = f.read()
    docs = [i.split("<DOC>")[1] for i in filter(lambda x:len(x)>10,txt.split('</DOC>'))]

    for doc in docs:
        try:
            id = doc.split("<DOCNO>")[1].split('</DOCNO>')[0]
            text = doc.split("<TEXT>")[1].split('</TEXT>')[0]
            docs_json[id] = text
        except:
            errors+=1
print("Reading APWSJ dataset, Errors : ",errors)

docs_json = {k.strip():v.strip() for k,v in docs_json.items()}

Errors :  0


In [2]:
import os
from collections import defaultdict

topic_to_doc_file = "./dataset/apwsj/NoveltyData/apwsj.qrels"
with open(topic_to_doc_file,'r') as f:
    topic_to_doc = f.read()
topic_doc = [(i.split(' 0 ')[1][:-2],i.split(' 0 ')[0]) for i in topic_to_doc.split("\n")]
topics = "q101, q102, q103, q104, q105, q106, q107, q108, q109, q111, q112, q113, q114, q115, q116, q117, q118, q119, q120, q121, q123, q124, q125, q127, q128, q129, q132, q135, q136, q137, q138, q139, q141"
topic_list = topics.split(", ")
filterd_docid = [(k,v) for k,v in topic_doc if v in topic_list]

def crawl(red_dict,doc,crawled):
	ans=[]
	for cdoc in red_dict[doc]:
		ans.append(cdoc)
		if crawled[cdoc]==0:
			try:
				red_dict[cdoc]=crawl(red_dict,cdoc,crawled)
				crawled[cdoc]=1
				ans+=red_dict[cdoc]
			except:
				crawled[cdoc]=1
	return ans

wf = "./dataset/apwsj/redundancy_list_without_partially_redundant.txt"
topics_allowed="q101, q102, q103, q104, q105, q106, q107, q108, q109, q111, q112, q113, q114, q115, q116, q117, q118, q119, q120, q121, q123, q124, q125, q127, q128, q129, q132, q135, q136, q137, q138, q139, q141"
topics_allowed=topics_allowed.split(", ")
red_dict = dict()
allow_partially_redundant = 1
for line in open("./dataset/apwsj/NoveltyData/redundancy.apwsj.result","r"):
	tokens = line.split()
	if tokens[2]=="?":
		if allow_partially_redundant==1:
			red_dict[tokens[0]+"/"+tokens[1]]=[tokens[0]+"/"+i for i in tokens[3:]]
	else:
		red_dict[tokens[0]+"/"+tokens[1]]=[tokens[0]+"/"+i for i in tokens[2:]]
crawled=defaultdict(int)
for doc in red_dict:
	if crawled[doc]==0:
		red_dict[doc]=crawl(red_dict,doc,crawled)
		crawled[doc]=1
with open(wf,"w") as f:
	for doc in red_dict:
		if doc.split("/")[0] in topics_allowed:
			f.write(" ".join(doc.split("/")+[i.split("/")[1] for i in red_dict[doc]])+"\n")

write_file="./dataset/apwsj/novel_list_without_partially_redundant.txt"
topics = topic_list
doc_topic_dict=defaultdict(list)

for i in topic_doc:
    doc_topic_dict[i[0]].append(i[1])
docs_sorted = open("./dataset/apwsj/NoveltyData/apwsj88-90.rel.docno.sorted","r").read().splitlines()
sorted_doc_topic_dict = defaultdict(list)
for doc in docs_sorted:
	if len(doc_topic_dict[doc])>0:
		for t in doc_topic_dict[doc]:
			sorted_doc_topic_dict[t].append(doc)
redundant_dict= defaultdict(lambda: defaultdict(int))
for line in open("./dataset/apwsj/redundancy_list_without_partially_redundant.txt","r"):
	tokens=line.split()
	redundant_dict[tokens[0]][tokens[1]]=1
novel_list=[]
for topic in topics:
	if topic in topics_allowed:
		for i in range(len(sorted_doc_topic_dict[topic])):
			if redundant_dict[topic][sorted_doc_topic_dict[topic][i]]!=1:
				if i>0:
					# take at most 5 latest docs in case of novel
					novel_list.append(" ".join([topic,sorted_doc_topic_dict[topic][i]]+sorted_doc_topic_dict[topic][max(0,i-5):i]))
with open(write_file,"w") as f:
	f.write("\n".join(novel_list))

In [3]:
# Novel cases
novel_docs="./dataset/apwsj/novel_list_without_partially_redundant.txt"
with open(novel_docs,'r') as f:
    novel_doc_list = [i.split() for i in f.read().split("\n")]
# Redundant cases
red_docs="./dataset/apwsj/redundancy_list_without_partially_redundant.txt"
with open(red_docs,'r') as f:
    red_doc_list = [i.split() for i in f.read().split("\n")]
red_doc_list = filter(lambda x:len(x)>0,red_doc_list)
novel_doc_list = filter(lambda x:len(x)>0,novel_doc_list)

dataset = []
s_not_found=0
t_not_found=0
for i in novel_doc_list:
    target_id = i[1]
    source_ids = i[2:]
    if target_id in docs_json.keys():
        data_inst = {}
        data_inst["target"] = docs_json[target_id]
        data_inst["source"] = ""
        for source_id in source_ids:
            if source_id in docs_json.keys():
                data_inst["source"]+=docs_json[source_id]+'\n'
        data_inst["label"] = 1
    if data_inst["source"]!="":
        dataset.append(data_inst)


for i in red_doc_list:
    target_id = i[1]
    source_ids = i[2:]
    if target_id in docs_json.keys():
        data_inst = {}
        data_inst["target"] = docs_json[target_id]
        data_inst["source"] = ""
        for source_id in source_ids:
            if source_id in docs_json.keys():
                data_inst["source"]+=docs_json[source_id]+'\n'
        data_inst["label"] = 0
    if data_inst["source"]!="":
        dataset.append(data_inst)

dataset_json = {}
for i in range(len(dataset)):
    dataset_json[i] = dataset[i]

    


In [7]:
import json
dataset_path="./dataset/apwsj/apwsj_dataset.json"
with open(dataset_path,'w') as f:
    json.dump(dataset_json,f)


In [2]:
apwsj_data_path = "./dataset/apwsj/apwsj_dataset.json"

In [5]:
import json
with open(apwsj_data_path,'r') as f:
    dataset = json.load(f)

In [4]:
from datamodule import APWSJDataModule
from utils import load_bilstm_encoder, load_attn_encoder
encoder, Lang = load_attn_encoder()

In [5]:
data_module = APWSJDataModule(batch_size=32)

In [6]:
data_module.prepare_data(Lang,100)

Data Size  5634
train_samples:  4507


In [22]:
data_module.APWSJ_data.__getitem__(100)[1].shape

(168, 100)

In [2]:
from dataloaders import APWSJDataset

In [7]:
dataset = APWSJDataset()
# dataset1 = APWSJDataset()

In [46]:
dataset.org = [
            list(
                filter(
                    lambda x: x != "" and x != " ",
                    [lang.preprocess_sentence(j) for j in nltk.sent_tokenize(i)],
                )
            )
            for i in dataset.org
        ]

In [4]:
lang = Lang

dataset.encode_lang(lang)

In [9]:
x = 0
for i in dataset.org:
    x+=1
    if i.shape[0]==0:
        print(x)


111
692
693
1069
1274
2602
3484
3485
4472
4473
